파이토치에서는 데이터셋을 좀 더 쉽게 다룰 수 있도록 **torch.utils.data.Dataset과 torch.utils.data.DataLoader**를 제공합니다. 이를 사용하면 미니 배치 학습, 데이터 셔플, 병렬 처리까지 간단히 수행할 수 있습니다. 

기본적인 사용 방법은 **Dataset을 정의**하고, 이를 **DataLoader에 전달**하는 것입니다.

그런데 torch.utils.data.Dataset을 상속받아 직접 커스텀 데이터셋(Custom Dataset)을 만드는 경우도 있습니다. 

>- torch.utils.data.Dataset : 파이토치에서 데이터셋을 제공하는 추상 클래스

Dataset을 상속받아 다음 메소드들을 오버라이드 하여 커스텀 데이터셋을 만들어 보겠습니다.

## 1.커스텀 데이터셋(Custom Dataset)

커스텀 데이터셋을 만들 때, 가장 기본적인 뼈대는 아래와 같다. 여기서 필요한 기본적인 define은 3개이다.

```
  class CustomDataset(torch.utils.data.Dataset): 

    def __init__(self):
    데이터셋의 **전처리** 를 해주는 부분

    def __len__(self):
    **데이터셋의 길이**. 즉, 총 샘플의 수를 적어주는 부분

    def __getitem__(self, idx): 
    데이터셋에서 **특정 1개의 샘플을 가져오는** 함수
```

- len(dataset)을 했을 때 데이터셋의 크기를 리턴할 len
- dataset[i]을 했을 때 i번째 샘플을 가져오도록 하는 인덱싱을 위한 get_item

## 2.커스텀 데이터셋(Custom Dataset)으로 선형 회귀 구현하기

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
# Dataset 상속
class CustomDataset(Dataset): 
  def __init__(self):
    self.x_data = [[73, 80, 75],
                   [93, 88, 93],
                   [89, 91, 90],
                   [96, 98, 100],
                   [73, 66, 70]]
    self.y_data = [[152], [185], [180], [196], [142]]

  # 총 데이터의 개수를 리턴
  def __len__(self): 
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx): 
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])
    return x, y

In [ ]:
dataset = CustomDataset()

# 데이터셋을 DataLoader 에 전달
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [ ]:
model = torch.nn.Linear(3,1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

In [ ]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):

    x_train, y_train = samples

    prediction = model(x_train) # H(x) 계산
    cost = F.mse_loss(prediction, y_train)  # cost 계산

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()))

Epoch    0/20 Batch 1/3 Cost: 0.628014
Epoch    0/20 Batch 2/3 Cost: 0.113793
Epoch    0/20 Batch 3/3 Cost: 0.008596
Epoch    1/20 Batch 1/3 Cost: 0.203202
Epoch    1/20 Batch 2/3 Cost: 0.756523
Epoch    1/20 Batch 3/3 Cost: 0.004933
Epoch    2/20 Batch 1/3 Cost: 0.161877
Epoch    2/20 Batch 2/3 Cost: 0.590848
Epoch    2/20 Batch 3/3 Cost: 0.283281
Epoch    3/20 Batch 1/3 Cost: 0.084428
Epoch    3/20 Batch 2/3 Cost: 0.031898
Epoch    3/20 Batch 3/3 Cost: 1.384651
Epoch    4/20 Batch 1/3 Cost: 0.252999
Epoch    4/20 Batch 2/3 Cost: 0.457416
Epoch    4/20 Batch 3/3 Cost: 0.594411
Epoch    5/20 Batch 1/3 Cost: 0.674270
Epoch    5/20 Batch 2/3 Cost: 0.208846
Epoch    5/20 Batch 3/3 Cost: 0.107246
Epoch    6/20 Batch 1/3 Cost: 0.724833
Epoch    6/20 Batch 2/3 Cost: 0.118906
Epoch    6/20 Batch 3/3 Cost: 0.009716
Epoch    7/20 Batch 1/3 Cost: 0.666923
Epoch    7/20 Batch 2/3 Cost: 0.085763
Epoch    7/20 Batch 3/3 Cost: 0.017080
Epoch    8/20 Batch 1/3 Cost: 0.069969
Epoch    8/20 Batch 2/3 C

In [ ]:
# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y) 

훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[150.9053]], grad_fn=<AddmmBackward0>)
